```
For example, if the spinlock were to step 3 times per insert, the circular buffer would begin to evolve like this (using parentheses to mark the current position after each iteration of the algorithm):

(0), the initial state before any insertions.
0 (1): the spinlock steps forward three times (0, 0, 0), and then inserts the first value, 1, after it. 1 becomes the current position.
0 (2) 1: the spinlock steps forward three times (0, 1, 0), and then inserts the second value, 2, after it. 2 becomes the current position.
0  2 (3) 1: the spinlock steps forward three times (1, 0, 2), and then inserts the third value, 3, after it. 3 becomes the current position.
And so on:

0  2 (4) 3  1
0 (5) 2  4  3  1
0  5  2  4  3 (6) 1
0  5 (7) 2  4  3  6  1
0  5  7  2  4  3 (8) 6  1
0 (9) 5  7  2  4  3  8  6  1
```

In [13]:
class CircularBuffer(object):
    def __init__(self):
        self.buffer = [0]
        self.position = 0
        
    def step_forward(self, steps):
        self.position = (self.position + steps) % len(self.buffer)
    
    def insert(self, num):
        self.buffer.insert(self.position + 1, num)

    @property
    def state(self):
        return list(self.buffer)        

In [14]:
c = CircularBuffer()
move = lambda x: c.step_forward(3) or c.insert(x) or c.step_forward(1)
move(1)
assert c.state == [0, 1]
move(2)
assert c.state == [0, 2, 1]
move(3)
assert c.state == [0, 2, 3, 1]
move(4)
assert c.state == [0, 2, 4, 3, 1]
move(5)
assert c.state == [0, 5, 2, 4, 3, 1]
move(6)
assert c.state == [0, 5, 2, 4, 3, 6, 1]
move(7)
assert c.state == [0, 5, 7, 2, 4, 3, 6, 1]
move(8)
assert c.state == [0, 5, 7, 2, 4, 3, 8, 6, 1]
move(9)
assert c.state == [0, 9, 5, 7, 2, 4, 3, 8, 6, 1]

In [15]:
def play(num_steps, num_iterations, buffer_cls=CircularBuffer, progress_every=10000000):
    buffer = buffer_cls()
    for i in range(num_iterations):
        if i % progress_every == 0:
            print('Iteration: ', i)
        buffer.step_forward(num_steps)
        buffer.insert(i + 1)
        buffer.step_forward(1)
    return buffer.state    

In [16]:
state = play(3, 2017)

Iteration:  0


In [17]:
i = state.index(2017)
state[i-3:i+3]

[1512, 1134, 151, 2017, 638, 1513]

In [18]:
state = play(354, 2017)
i = state.index(2017)
state[i-3:i+3]

Iteration:  0


[162, 240, 1252, 2017, 2000, 1072]

In [19]:
state = play(354, 50000000)

Iteration:  0


KeyboardInterrupt: 

In [20]:
class DummyCircularBuffer(object):
    def __init__(self):
        self.buffer_size = 1
        self.position = 0
        self.buffer = None
        
    def step_forward(self, steps):
        self.position = (self.position + steps) % self.buffer_size
    
    def insert(self, num):
        self.buffer_size += 1
        if self.position == 0:
            self.buffer = num

    @property
    def state(self):
        return self.buffer

In [21]:
play(3, 2017)[0:2]

Iteration:  0


[0, 1226]

In [22]:
play(3, 2017, DummyCircularBuffer)

Iteration:  0


1226

In [ ]:
play(354, 50000000, DummyCircularBuffer)

Iteration:  0
Iteration:  10000000
Iteration:  20000000
Iteration:  30000000


In [12]:
state

10242889